In [1]:
#cell-1
!pip install wandb --quiet
import wandb
from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()
wandb_api = user_secrets.get_secret("WANDB_API_KEY")

wandb.login(key=wandb_api)


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: 22f3000982 (22f3000982-indian-institute-of-technology-madras) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [2]:
wandb.init(
    project="2025-sep-dl-genai-project",   
    name="roberta_multilabel_v3",          
    config={
        "model": "roberta-base",
        "epochs": 8,
        "batch_size": 16,
        "learning_rate": 2e-5,
        "weight_decay": 0.02,
        "warmup_ratio": 0.1,
        "early_stop_patience": 2,
    }
)


In [3]:
import pandas as pd
import numpy as np
import torch
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split

In [4]:
!pip install -q transformers datasets torch scikit-learn accelerate

import pandas as pd
import numpy as np
import torch
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments
)
from datasets import Dataset


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 56.9 MB/s eta 0:00:0000:01m00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 51.9 MB/s eta 0:00:0000:01:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 54.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 72.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 17.2 MB/s eta 0:00:00
ERROR: pip's dependency

2025-11-30 02:55:30.116531: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764471330.295387      38 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764471330.350134      38 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [5]:

device_name = torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU"
print("device", device_name)


device Tesla T4


In [16]:
train_df = pd.read_csv("/kaggle/input/2025-sep-dl-gen-ai-project/train.csv")
test_df = pd.read_csv("/kaggle/input/2025-sep-dl-gen-ai-project/test.csv")

print(train_df.head())
print("Train shape:", train_df.shape)
print("Test shape:", test_df.shape)


   id                                               text  anger  fear  joy  \
0   0  the dentist that did the work apparently did a...      1     0    0   
1   1  i'm gonna absolutely ~~suck~~ be terrible duri...      0     1    0   
2   2  bridge: so leave me drowning calling houston, ...      0     1    0   
3   3  after that mess i went to see my now ex-girlfr...      1     1    0   
4   4  as he stumbled i ran off, afraid it might some...      0     1    0   

   sadness  surprise                    emotions  
0        1         0         ['anger' 'sadness']  
1        1         0          ['fear' 'sadness']  
2        1         0          ['fear' 'sadness']  
3        1         0  ['anger' 'fear' 'sadness']  
4        0         0                    ['fear']  
Train shape: (6827, 8)
Test shape: (1707, 2)


In [17]:
import pandas as pd
import string
import nltk
from nltk.corpus import stopwords
from collections import Counter

nltk.download('stopwords')

train_df = pd.read_csv("/kaggle/input/2025-sep-dl-gen-ai-project/train.csv")

train_df['clean_text'] = train_df['text'].str.lower()


before_count = train_df['clean_text'].str.len().sum()


translator = str.maketrans('', '', string.punctuation)
train_df['clean_text'] = train_df['clean_text'].apply(lambda x: x.translate(translator))

after_count = train_df['clean_text'].str.len().sum()


reduction_percent = ((before_count - after_count) / before_count) * 100
print(f" Percentage reduction in character count: {reduction_percent:.2f}%")


words = []
for text in train_df['clean_text']:
    words.extend(text.split())

unique_words = set(words)
print("Total unique words:", len(unique_words))

stop_words = set(stopwords.words('english'))
common_stopwords = unique_words.intersection(stop_words)

stopword_percent = (len(common_stopwords) / len(unique_words)) * 100
print(f" Percentage of unique words that are stop words: {stopword_percent:.2f}%")

filtered_words = [w for w in words if w not in stop_words]
word_freq = Counter(filtered_words)
most_common_words = word_freq.most_common(10)

print("\nTop 10 most frequent non_stop word:")
for i, (word, count) in enumerate(most_common_words, 1):
    print(f"{i}. {word} - {count}")

fifth_most_freq_word = most_common_words[4][0]
print(f"\n 5th most frequent word (excluding stop words): '{fifth_most_freq_word}'")


 Percentage reduction in character count: 3.26%
Total unique words: 8365
 Percentage of unique words that are stop words: 1.54%

Top 10 most frequent non_stop word:
1. head - 539
2. eyes - 438
3. like - 394
4. back - 365
5. heart - 334
6. one - 323
7. face - 293
8. get - 291
9. time - 271
10. still - 271

 5th most frequent word (excluding stop words): 'heart'


[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [18]:
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import Dataset

labels = ['anger', 'fear', 'joy', 'sadness', 'surprise']

print("Initial rows:", train_df.shape[0])
print("Missing text rows in train:", train_df['text'].isnull().sum())
train_df = train_df.dropna(subset=['text']).reset_index(drop=True)
print("Rows after dropping null-text:", train_df.shape[0])

for l in labels:
    train_df[l] = train_df[l].astype(int)

if 'emotions' in train_df.columns:
    train_df = train_df.drop(columns=['emotions'])

print("\nLabel counts (whole train):")
print(train_df[labels].sum())

n_labels = train_df[labels].sum(axis=1)
print("\nDistribution of #labels per sample (whole train):")
print(n_labels.value_counts().sort_index())

train_data, val_data = train_test_split(train_df, test_size=0.10, random_state=42, shuffle=True)

print(f"\nTrain rows: {len(train_data)}    |    Val rows: {len(val_data)}")

print("\nLabel counts (train split):")
print(train_data[labels].sum())
print("\nLabel counts (val split):")
print(val_data[labels].sum())

train_dataset = Dataset.from_pandas(train_data.reset_index(drop=True))
val_dataset = Dataset.from_pandas(val_data.reset_index(drop=True))

print("\nExample training sample (dict):")
print(train_dataset[0])

print("\nTest missing text rows:", test_df['text'].isnull().sum())


Initial rows: 6827
Missing text rows in train: 0
Rows after dropping null-text: 6827

Label counts (whole train):
anger        808
fear        3860
joy         1660
sadness     2171
surprise    1999
dtype: int64

Distribution of #labels per sample (whole train):
0     676
1    2743
2    2587
3     706
4     112
5       3
Name: count, dtype: int64

Train rows: 6144    |    Val rows: 683

Label counts (train split):
anger        728
fear        3468
joy         1497
sadness     1961
surprise    1798
dtype: int64

Label counts (val split):
anger        80
fear        392
joy         163
sadness     210
surprise    201
dtype: int64

Example training sample (dict):
{'id': 2276, 'text': 'it leaves both my hands free to be able to type.', 'anger': 0, 'fear': 0, 'joy': 1, 'sadness': 0, 'surprise': 0, 'clean_text': 'it leaves both my hands free to be able to type'}

Test missing text rows: 0


In [19]:
from transformers import AutoTokenizer

MODEL_NAME = "roberta-base"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

def tokenize_function(example):
    return tokenizer(
        example["text"],
        truncation=True,
        padding="max_length",
        max_length=256
    )

train_tokenized = train_dataset.map(tokenize_function, batched=True)
val_tokenized = val_dataset.map(tokenize_function, batched=True)

label_cols = ['anger', 'fear', 'joy', 'sadness', 'surprise']

train_tokenized = train_tokenized.rename_column("anger", "labels_anger")
train_tokenized = train_tokenized.rename_column("fear", "labels_fear")
train_tokenized = train_tokenized.rename_column("joy", "labels_joy")
train_tokenized = train_tokenized.rename_column("sadness", "labels_sadness")
train_tokenized = train_tokenized.rename_column("surprise", "labels_surprise")

val_tokenized = val_tokenized.rename_column("anger", "labels_anger")
val_tokenized = val_tokenized.rename_column("fear", "labels_fear")
val_tokenized = val_tokenized.rename_column("joy", "labels_joy")
val_tokenized = val_tokenized.rename_column("sadness", "labels_sadness")
val_tokenized = val_tokenized.rename_column("surprise", "labels_surprise")

train_tokenized.set_format("torch", columns=["input_ids", "attention_mask"] + [f"labels_{l}" for l in label_cols])
val_tokenized.set_format("torch", columns=["input_ids", "attention_mask"] + [f"labels_{l}" for l in label_cols])

print("Tokenization complete!")
print("Train tokenized sample:")
print(train_tokenized[0])


Map:   0%|          | 0/6144 [00:00<?, ? examples/s]

Map:   0%|          | 0/683 [00:00<?, ? examples/s]

Tokenization complete!
Train tokenized sample:
{'labels_anger': tensor(0), 'labels_fear': tensor(0), 'labels_joy': tensor(1), 'labels_sadness': tensor(0), 'labels_surprise': tensor(0), 'input_ids': tensor([   0,  405, 3607,  258,  127, 1420,  481,    7,   28,  441,    7, 1907,
           4,    2,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
  

In [21]:
import torch
from torch import nn
from transformers import AutoModel

class RobertaMultiLabel(nn.Module):
    def __init__(self, model_name, num_labels=5):
        super(RobertaMultiLabel, self).__init__()
        self.roberta = AutoModel.from_pretrained(model_name)
        self.dropout = nn.Dropout(0.3)
        self.classifier = nn.Linear(self.roberta.config.hidden_size, num_labels)

    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.roberta(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs[0][:, 0]  # CLS token output
        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)

        loss = None
        if labels is not None:
            pos_weights = torch.tensor([1.2, 0.9, 1.1, 1.0, 1.3], device=logits.device)
            loss_fct = nn.BCEWithLogitsLoss(pos_weight=pos_weights)
            loss = loss_fct(logits, labels.float())

        return {"loss": loss, "logits": logits}


def collate_fn(batch):
    """
    Custom collate function to combine multiple emotion label columns into one tensor.
    """
    input_ids = torch.stack([torch.tensor(x["input_ids"]).clone().detach() for x in batch])
    attention_mask = torch.stack([torch.tensor(x["attention_mask"]).clone().detach() for x in batch])

    labels = torch.stack([
        torch.tensor([
            x["labels_anger"],
            x["labels_fear"],
            x["labels_joy"],
            x["labels_sadness"],
            x["labels_surprise"]
        ]).clone().detach().float()
        for x in batch
    ])

    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }



In [22]:
print(train_tokenized[0].keys())


dict_keys(['labels_anger', 'labels_fear', 'labels_joy', 'labels_sadness', 'labels_surprise', 'input_ids', 'attention_mask'])


In [ ]:
#cell-6
wandb.init(
    project="2025-sep-dl-genai-project",
    name="roberta_multilabel_stable",
    config={
        "model": "roberta-base",
        "epochs": 8,
        "batch_size": 16,
        "learning_rate": 2e-5,
        "weight_decay": 0.02,
        "warmup_ratio": 0.1,
        "early_stop_patience": 2
    }
)

# ,,--------------- Model----------------------------
class RobertaMultiLabel(nn.Module):
    def __init__(self, model_name, num_labels=5):
        super().__init__()
        from transformers import AutoModel
        self.roberta = AutoModel.from_pretrained(model_name)
        self.dropout = nn.Dropout(0.2)      # dropped from 0.4 → 0.2 (better)
        self.classifier = nn.Linear(self.roberta.config.hidden_size, num_labels)

    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.roberta(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.last_hidden_state[:, 0]
        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)
        return {"logits": logits}


def collate_fn(batch):
    input_ids = torch.stack([x["input_ids"] for x in batch])
    attention_mask = torch.stack([x["attention_mask"] for x in batch])
    labels = torch.stack([
        torch.stack([
            x["labels_anger"], x["labels_fear"], x["labels_joy"],
            x["labels_sadness"], x["labels_surprise"]
        ])
        for x in batch
    ])
    return {"input_ids": input_ids, "attention_mask": attention_mask, "labels": labels}


def evaluate_model(model, dataloader, loss_fn, thresholds):
    model.eval()
    total_loss = 0.0
    all_logits, all_labels = [], []

    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device).float()

            logits = model(input_ids=input_ids, attention_mask=attention_mask)["logits"]
            loss = loss_fn(logits, labels)
            total_loss += loss.item()

            all_logits.append(torch.sigmoid(logits).cpu().numpy())
            all_labels.append(labels.cpu().numpy())

    avg_loss = total_loss / len(dataloader)

    all_logits = np.vstack(all_logits)
    all_labels = np.vstack(all_labels)

    thresholds = np.array(thresholds)
    preds = (all_logits >= thresholds).astype(int)

    f1_macro = f1_score(all_labels, preds, average="macro")
    f1_micro = f1_score(all_labels, preds, average="micro")
    acc = accuracy_score(all_labels, preds)

    try:
        roc = roc_auc_score(all_labels, all_logits, average="macro")
    except:
        roc = 0.0

    return {
        "loss": avg_loss,
        "f1_macro": f1_macro,
        "f1_micro": f1_micro,
        "accuracy": acc,
        "roc_auc": roc
    }



MODEL_NAME = "roberta-base"
num_epochs = 8
per_device_train_batch_size = 16
per_device_eval_batch_size = 16
grad_accum_steps = 2
learning_rate = 2e-5
weight_decay = 0.02
warmup_ratio = 0.1
max_grad_norm = 1.0
early_stop_patience = 2



train_loader = DataLoader(
    train_tokenized,
    batch_size=per_device_train_batch_size,
    shuffle=True,
    collate_fn=collate_fn
)
val_loader = DataLoader(
    val_tokenized,
    batch_size=per_device_eval_batch_size,
    shuffle=False,
    collate_fn=collate_fn
)

model = RobertaMultiLabel(MODEL_NAME, num_labels=5).to(device)
optimizer = AdamW(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

num_training_steps = len(train_loader) // grad_accum_steps * num_epochs
scheduler = get_cosine_schedule_with_warmup(
    optimizer,
    num_warmup_steps=int(num_training_steps * warmup_ratio),
    num_training_steps=num_training_steps
)

# ===== POS WEIGHTS FROM CELL-A =====
pos_weights = torch.tensor(pos_weights_list, device=device)
loss_fn = nn.BCEWithLogitsLoss(pos_weight=pos_weights)

print("Using pos_weights:", pos_weights)
print("Using thresholds:", per_label_thresholds)


# ===== TRAIN LOOP =====
best_f1 = 0
epochs_no_improve = 0
global_step = 0

for epoch in range(1, num_epochs + 1):
    model.train()
    running_loss = 0.0
    start_time = time.time()
    optimizer.zero_grad()

    pbar = tqdm(enumerate(train_loader), total=len(train_loader), desc=f"Epoch {epoch}/{num_epochs}")
    for step, batch in pbar:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device).float()

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs["logits"]
        loss = loss_fn(logits, labels) / grad_accum_steps
        loss.backward()
        running_loss += loss.item() * grad_accum_steps

        if (step + 1) % grad_accum_steps == 0:
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()
            global_step += 1

        pbar.set_postfix({
            "loss": f"{running_loss/(step+1):.4f}",
            "lr": scheduler.get_last_lr()[0]
        })

    # ---- UPDATED EVAL CALL ----
    val_metrics = evaluate_model(
        model,
        val_loader,
        loss_fn,
        thresholds=per_label_thresholds
    )

    elapsed = time.time() - start_time

    print(f"\nEpoch {epoch}/{num_epochs} — time: {elapsed/60:.2f} min")
    print(f"  Train loss: {running_loss/len(train_loader):.4f}")
    print(f"  Val F1-macro: {val_metrics['f1_macro']:.4f} | F1-micro: {val_metrics['f1_micro']:.4f} | ROC-AUC: {val_metrics['roc_auc']:.4f}")

    wandb.log({
        "epoch": epoch,
        "train_loss": running_loss / len(train_loader),
        "val_loss": val_metrics["loss"],
        "f1_macro": val_metrics["f1_macro"],
        "f1_micro": val_metrics["f1_micro"],
        "roc_auc": val_metrics["roc_auc"],
        "accuracy": val_metrics["accuracy"],
        "learning_rate": scheduler.get_last_lr()[0]
    })

    if val_metrics["f1_macro"] > best_f1:
        best_f1 = val_metrics["f1_macro"]
        epochs_no_improve = 0
        best_dir = "./roberta_emotion_best"
        os.makedirs(best_dir, exist_ok=True)
        torch.save(model.state_dict(), os.path.join(best_dir, "pytorch_model.bin"))
        wandb.save(os.path.join(best_dir, "pytorch_model.bin"))
        print(f" New best model saved! F1-macro={best_f1:.4f}")
    else:
        epochs_no_improve += 1
        print(f"  No improvement for {epochs_no_improve} epoch(s).")

    if epochs_no_improve >= early_stop_patience:
        print("Early stopping triggered.")
        break

print(f"\nTraining complete. Best F1-macro: {best_f1:.4f}")

wandb.finish()


In [24]:
#cell-8
import torch
import numpy as np
import pandas as pd
from tqdm import tqdm
from datasets import Dataset

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Tokenize test data 
test_dataset = Dataset.from_pandas(test_df)
test_tokenized = test_dataset.map(
    lambda x: tokenizer(x["text"], truncation=True, padding="max_length", max_length=256),
    batched=True
)
test_tokenized.set_format("torch", columns=["input_ids", "attention_mask"])


best_model_path = "./roberta_emotion_best/pytorch_model.bin"
model.load_state_dict(torch.load(best_model_path, map_location=device))
model.to(device)
model.eval()
THRESHOLD = 0.5   

print("Using global threshold:", THRESHOLD)

preds = []
test_loader = torch.utils.data.DataLoader(test_tokenized, batch_size=16, shuffle=False)

for batch in tqdm(test_loader, desc="Predicting"):
    with torch.no_grad():
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs["logits"]
        preds.append(torch.sigmoid(logits).cpu().numpy())

preds = np.vstack(preds)  

# Convert to binary labels using global threshold
binary_preds = (preds >= THRESHOLD).astype(int)

submission = pd.DataFrame(
    binary_preds,
    columns=["anger", "fear", "joy", "sadness", "surprise"]
)
submission.insert(0, "id", test_df["id"])

submission.to_csv("submission.csv", index=False)
print("submission.csv saved successfully!")
submission.head()


Map:   0%|          | 0/1707 [00:00<?, ? examples/s]

Using global threshold: 0.5


Predicting: 100%|██████████| 107/107 [00:21<00:00,  5.03it/s]

submission.csv saved successfully!


,id,anger,fear,joy,sadness,surprise
0,0,1,1,0,0,0
1,1,0,0,0,0,0
2,2,1,1,0,0,1
3,3,0,1,0,0,0
4,4,0,1,0,0,1


In [17]:

# import os

# submission_path = "/kaggle/working/submission.csv"
# submission.to_csv(submission_path, index=False)

# print(f"submission.csv saved at: {submission_path}")
# print("Files in working dir:", os.listdir("/kaggle/working"))


submission.csv saved at: /kaggle/working/submission.csv
Files in working dir: ['wandb', '.virtual_documents', 'submission.csv', 'hf_model', 'roberta_emotion_best']


# Milestone 3 below 

In [ ]:
# import math

# # Q1: Sigmoid output for z = 1.5
# z = 1.5
# sigmoid = 1 / (1 + math.exp(-z))
# print("1️ Sigmoid(1.5):", round(sigmoid, 3))  # 0.818

# # Q2: Total parameters in BERT-BASE
# bert_base_params = 110_000_000
# print("2️ BERT-BASE total parameters:", bert_base_params, "(~110M)")

# # Q3: Binary Cross Entropy Loss (y=1, p=0.9)
# y, p = 1, 0.9
# bce_loss = - (y * math.log(p) + (1 - y) * math.log(1 - p))
# print("3️ Binary Cross-Entropy Loss:", round(bce_loss, 3))  # 0.105

# # Q4: Special tokens added by BERT for single sequence
# special_tokens = 2  # [CLS], [SEP]
# print("4️ Special tokens added:", special_tokens)

# # Q5: Number of encoder layers in BERT-BASE
# encoder_layers = 12
# print("5️ Encoder layers in BERT-BASE:", encoder_layers)

# # Q6: Recommended Adam variant for fine-tuning Transformers
# adam_variant = "AdamW"
# print("6️ Recommended Adam variant:", adam_variant)

# # Q7: Nonlinearity used in Transformer blocks
# activation_fn = "GELU"
# print("7️ Activation function in Transformer blocks:", activation_fn)

# # Q8: Optimal learning rate range for fine-tuning Transformers
# lr_range = "2e-5 to 5e-5"
# print("8️ Typical fine-tuning learning rate range:", lr_range)

# # Q9: Dropout rate in BERT classification head (default for BERT-base)
# dropout_rate = 0.1
# print("9️ Dropout rate in BERT classification head:", dropout_rate)

# # Q10: Practical upper limit for batch size on Colab T4 GPU (16GB VRAM)
# # For BERT-base (sequence length 128-256)
# batch_size_limit = 32
# print(" Practical upper batch size limit (Colab T4):", batch_size_limit)


# Milestone 4 below

In [ ]:
# import re
# import pandas as pd
# from collections import Counter

# train_path = "/kaggle/input/2025-sep-dl-gen-ai-project/train.csv"
# df = pd.read_csv(train_path)

# def tokenize(x):
#     return re.findall(r"[A-Za-z0-9']+", str(x).lower())

# counter = Counter()
# for t in df["text"].fillna(""):
#     counter.update(tokenize(t))

# # remove rare words (min_freq = 3)
# counter = Counter({w:c for w,c in counter.items() if c >= 3})

# PAD = "<pad>"
# UNK = "<unk>"
# itos = [PAD, UNK] + sorted(counter.keys())
# stoi = {w:i for i,w in enumerate(itos)}

# print("Vocabulary Size:", len(itos))
# print("Padding index:", stoi[PAD])
# print("Unknown index:", stoi[UNK])

# print("\nIndices:")
# print("happy:", stoi.get("happy", None))
# print("alone:", stoi.get("alone", None))
# print("sad:", stoi.get("sad", None))
